In [1]:
""" Ch8_03 """

""" 영화 리뷰 불러오기 """
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding = 'utf-8')
df.head(5)

""" 함수 정의 및 데이터 분할 """
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords

porter = PorterStemmer()
tfidf = TfidfTransformer(use_idf = True, norm = 'l2', smooth_idf = True)
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer(text) :
    return text.split()

def tokenizer_porter(text) : 
    return [porter.stem(word) for word in text.split()]

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

""" 5겹 교차 검증으로 로지스틱 회귀 매개변수 조합 찾기 """
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(solver='liblinear', random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

print('최적의 매개변수 조합 : %s' % gs_lr_tfidf.best_params_)

print('CV 정확도 : %.3f' % gs_lr_tfidf.best_score_)

clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도 : %.3f ' % clf.score(X_test, y_test))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Artyrie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed: 41.9min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 55.6min finished


최적의 매개변수 조합 : {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000001C80A0616A8>}
CV 정확도 : 0.888
테스트 정확도 : 0.890 


In [2]:
""" Ch8_04 """

""" 데이터 정제 및 토큰화 """
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

def tokenizer(text) : 
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

""" 문서를 하나씩 읽어서 반환 """

def stream_docs(path) :
    with open(path, 'r', encoding = 'utf-8') as csv : 
        next(csv)
        # 헤더 넘기기
        for line in csv : 
            text, label = line[:-3], int(line[-2])
            yield text, label

next(stream_docs(path = 'movie_data.csv'))

""" 지정한 만큼 문서를 반환 """

def get_minibatch(doc_stream, size) : 
    docs, y = [], []
    try : 
        for _ in range(size) : 
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration : 
        pass
    return docs, y

""" 데이터 종류에 상관없는 HashingVectorizer"""
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error = 'ignore',
                        n_features = 2**21,
                         # 해시 충돌 가능성을 줄임 => 로지스틱 회귀 모델의 가중치 개수도 늘어남
                        preprocessor = None,
                        tokenizer = tokenizer)
clf = SGDClassifier(loss = 'log', random_state = 1, max_iter = 1)
doc_stream = stream_docs(path = 'movie_data.csv')

""" 외부 메모리 학습 """
import pyprind

pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45) : 
    X_train, y_train = get_minibatch(doc_stream, size = 1000)
    if not X_train : 
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes = classes)
    pbar.update()
    
""" 모델 테스트 하기 """

X_test, y_test = get_minibatch(doc_stream, size = 5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

clf.partial_fit(X_test, y_test)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


정확도: 0.876


SGDClassifier(loss='log', max_iter=1, random_state=1)

In [3]:
""" 어플에 쓰일 파일과 데이터 저장 """

import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')

if not os.path.exists(dest) : 
    os.makedirs(dest)

pickle.dump(stop,
           open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
           protocol = 4)
pickle.dump(clf,
           open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            # wb 매개변수로 이진 모드로 파일을 염 (pickle 때문)
           protocol = 4)
            # protocol 4 지정시 3.4 버전 이상에서 사용 가능

# HashingVectorizer는 학습 과정이 없어 직렬화가 필요 없다.
# 대신 객체를 임포트할 수 있도록 파이썬 스크립트가 필요하다.
# 해당 파일은 vectorizer.py 참고